① Tool Definitions + Messages
개발자가 사용할 수 있는 함수(예: get_weather(location))를 미리 정의
사용자가 질문: “What’s the weather in Paris?”

② Tool Calls  
모델이 질문을 보고 “아, 이건 get_weather("paris") 함수를 호출해야겠네”  
텍스트가 아니라 함수 호출 요청(JSON) 을 생성  

③ Execute Function Code  
실제 코드에서 get_weather("paris") 실행  
외부 API(OpenWeather 같은) 호출  

④ Results (All Prior Messages)  
함수 실행 결과가 다시 모델에게 전달  
모델은 이제 “파리의 온도 = 14도”라는 사실을 알게 됨  

⑤ Final Response  
모델이 사용자에게 자연어로 최종 답변 생성  
“It’s currently 14°C in Paris.”  

- LLM이 API를 직접 실행하는 게 아니라  
“어떤 함수를 호출할지 결정”만 하고  
실행은 개발자 코드,  
결과를 다시 받아 문장 생성  
👉 LLM + 외부 시스템 연동 구조  

In [13]:
from openai import OpenAI                 # OpenAI API를 사용하기 위한 클라이언트 클래스
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('openai_key')
OPENWEATHER_API_KEY = os.getenv('openweather_key')
OPENWEATHER_API_KEY

'c9c41a782930cfc8c78bdeb5584b16b0'

In [14]:
# OpenWeather API 호출로 부산 날씨 데이터 조회
import requests

city_name = "Busan"
units = 'metric'        # 단위 (metrics = 섭씨)
url = f"https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={OPENWEATHER_API_KEY}&units={units}"

response = requests.get(url)

data = response.json()
data

{'coord': {'lon': 129.0403, 'lat': 35.1028},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': -2.01,
  'feels_like': -6.44,
  'temp_min': -2.01,
  'temp_max': -2.01,
  'pressure': 1030,
  'humidity': 43,
  'sea_level': 1030,
  'grnd_level': 1025},
 'visibility': 10000,
 'wind': {'speed': 3.6, 'deg': 310},
 'clouds': {'all': 0},
 'dt': 1770079685,
 'sys': {'type': 1,
  'id': 8086,
  'country': 'KR',
  'sunrise': 1770070906,
  'sunset': 1770108802},
 'timezone': 32400,
 'id': 1838524,
 'name': 'Busan',
 'cod': 200}

In [15]:
weather_info={}
if response.status_code==200:
    weather_description = data['weather'][0]['description']
    temp = data['main']['temp']
    temp_feels_like = data['main']['feels_like']
    humidity = data['main']['humidity']
    
    weather_info = {
        'city' : city_name,
        'description' : weather_description,
        'temperature' : temp,
        'temperature_feels_like' : temp_feels_like,         # 체감온도
        'humidity' : humidity                               # 습도
    }
    
else:
    weather_info = {
        'city' : city_name,
        'description' : 'Not Found',
        'temperature' : 'Not Found',
        'temperature_feels_like' : 'Not Found',
        'humidity' : 'Not Found'                  
    }
    
weather_info

{'city': 'Busan',
 'description': 'clear sky',
 'temperature': -2.01,
 'temperature_feels_like': -6.44,
 'humidity': 43}

In [16]:
# 실패시 실패 원이 ㄴ확인용 디버깅
print("status_code :", response.status_code)
print("response json :", data)
print("message :", data.get("message","no message"))

status_code : 200
response json : {'coord': {'lon': 129.0403, 'lat': 35.1028}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': -2.01, 'feels_like': -6.44, 'temp_min': -2.01, 'temp_max': -2.01, 'pressure': 1030, 'humidity': 43, 'sea_level': 1030, 'grnd_level': 1025}, 'visibility': 10000, 'wind': {'speed': 3.6, 'deg': 310}, 'clouds': {'all': 0}, 'dt': 1770079685, 'sys': {'type': 1, 'id': 8086, 'country': 'KR', 'sunrise': 1770070906, 'sunset': 1770108802}, 'timezone': 32400, 'id': 1838524, 'name': 'Busan', 'cod': 200}
message : no message


In [17]:
# 날씨 API 호출 함수 : doc_string을 활용해 함수 설명
import requests
import json

def get_current_weather(city_name='Seoul', units='metric'):
    """
    OpenWeather API를 사용해서 사용자가 지정한 도시의 현재 날씨 정보를 가져오는 함수

    Args:
        - city: str 날씨정보를 가져올 도시 이름. **반드시 영문으로 작성하세요.**
            - 변환예시:
                - 서울 -> Seoul
                - 충남, 충청남도 -> Chungcheongnam-do
                - 부산 -> Busan
        - units: str 온도단위를 설정하는 문자열
            - metric(기본값: 섭씨, 미터)
            - imperial(화씨, 야드)
    Return:
        - str: json 형식으로 변환된 현재 날씨 정보
    """
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={OPENWEATHER_API_KEY}&units={units}"
    response = requests.get(url)

    weather_info={}
    if response.status_code==200:
        weather_description = data['weather'][0]['description']
        temp = data['main']['temp']
        temp_feels_like = data['main']['feels_like']
        humidity = data['main']['humidity']
        
        weather_info = {
            'city' : city_name,
            'description' : weather_description,
            'temperature' : temp,
            'temperature_feels_like' : temp_feels_like,         # 체감온도
            'humidity' : humidity                               # 습도
        }
        
    else:
        weather_info = {
            'city' : city_name,
            'description' : 'Not Found',
            'temperature' : 'Not Found',
            'temperature_feels_like' : 'Not Found',
            'humidity' : 'Not Found'                  
        }
        
    return json.dumps(weather_info)  # dict를 JSON 문자열로 변환해 반환

In [18]:
# llm이 사용할 함수(tool) 모음
tools_to_execute = {
    'get_current_weather' : get_current_weather      # 키 : LLM이 호출할 tool 이름, 값 : 실제 실행될 함수
}

## llm 준비

In [19]:
# get_current_weather 함수의 docstring(설명) 출력
print(tools_to_execute['get_current_weather'].__doc__)


    OpenWeather API를 사용해서 사용자가 지정한 도시의 현재 날씨 정보를 가져오는 함수

    Args:
        - city: str 날씨정보를 가져올 도시 이름. **반드시 영문으로 작성하세요.**
            - 변환예시:
                - 서울 -> Seoul
                - 충남, 충청남도 -> Chungcheongnam-do
                - 부산 -> Busan
        - units: str 온도단위를 설정하는 문자열
            - metric(기본값: 섭씨, 미터)
            - imperial(화씨, 야드)
    Return:
        - str: json 형식으로 변환된 현재 날씨 정보
    


In [22]:
# Function Calling으로 날씨 API를 호출해 답변하는 대화 루프
from pprint import pprint       # 리스트/딕셔너리를 보기 좋게 출력
from openai import OpenAI

client = OpenAI()

# 사용자 질문을 받아 tool 호출 여부를 판단하고 최종 답변까지 생성하는 함수
def run_conversation(user_prompt, model='gpt-4.1-mini'):
    messages = [
        # 시스템 프롬프트
        {"role" : "system", 'content' : "당신은 친절한 챗봇입니다. 사용자의 요구를 분석해 직접 대답하거나, 주어진 함수를 이용해 필요한 정보를 먼저 확보한 후 대답하세요."} ,
        {"role" : "user", 'content' : user_prompt}  # 유저 프롬프트
    ]
    # llm이 사용 가능한 function(tool)에 대한 메타데이터(스키마)
    tools = [
        {
            'type' : 'function',    # tool의 타입
            'function' : {
                'name' : 'get_current_weather',     # LLM이 호출할 함수 이름
                'description' : tools_to_execute['get_current_weather'].__doc__,    # 함수 설명(문서화된 문자열)
                'parameters' : {    # 함수의 인다 스키마(JSON Schma)
                    'type' : 'object',      # 인자 타입
                    'properties' : {        # 사용 가능한 파라미터 정의
                        'city_name' : {     # 첫 번쨰 파라미터
                            'type' : 'string',
                            'description' :   """도시이름(필수값). 반드시 영어로 작성하세요.    
                                        변환예시:
                                            - 서울 -> Seoul
                                            - 충남, 충청남도 -> Chungcheongnam-do
                                            - 경남, 경상남도 -> Gyeongsangnam-do
                                            - 전남, 전라남도 -> Jeollanam-do
                                            - 부산 -> Busan"""                      
                        },  # 도서명 입력 안내(LL<이 올바른 인자를 생성)
                        'units' : {
                            'type' : 'string',
                            'description' : """
                                온도단위를 설정하는 문자열
                                    - metric(기본값: 섭씨, 미터)
                                    - imperial(화씨, 야드)""",
                            'enum' : ['metric', 'imperial'] # 허용 값 제한
                        }
                    },
                    'required' : ['city_name']  # 필수 입력값
                }    
            }
        }    
    ]
    
    # 첫 번째 LLM 호출 (함수 호출 필요 여부 판단)
    response1 = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools      # LLM에게 제공할 tool 메타정보
    )
    
    response1_message = response1.choices[0].message         # LLM의 첫 응답 메시지
    response1_tool_calls = response1_message.tool_calls     # LLM이 생성한 tool 호출 목록
    
    if response1_tool_calls:    # tool 호출이 존재하면
        messages.append(response1_message)  # tool_call이 함꼐 담긴 assistant 메시지
        
        for tool_call in response1_tool_calls:      # 요청한 tool_call 목록을 순회
            function_name = tool_call.function.name # 호출할 함수 이름
            print(f'[tool] {function_name}을 호출합니다!')
            function_to_excute = tools_to_execute[function_name]        # 함수 이름으로 실제 함수 객체 조회
            function_args = json.loads(tool_call.function.arguments)     # LLM이 준 JSON 문자열 인자를 dict로 파싱
            function_response = function_to_excute(**function_args)     # 파싱된 인자를 언패킹 후 함수 실행
            
            messages.append({
                'role' : "tool",        # 역할 : tool(함수 결과)
                'tool_call_id' : tool_call.id,  # 어떤 tool_call에 대한 응답인지 연결하는 ID
                'name' : function_name,         # ㅅㄹ행한 함수 이름
                'content' : function_response   # 함수 실행 결과(JSON 문자열)
            })
            pprint(messages)    # 현재까지의 모든 messages 확인
        
        # tool 결과를 포함하거나 리스트로 두번쨰 LLM을 호출
        response2 = client.chat.completions.create(
            model = model,
            messages = messages
        )
        return response2.choices[0].message.content # 최종 지연이 답변 변환
    # 호출이 없으면 첫 응답을 그대로 반환
    else:
        return response1_message.content

In [23]:
run_conversation("오늘 서울 날씨는 어때?")

[tool] get_current_weather을 호출합니다!
[{'content': '당신은 친절한 챗봇입니다. 사용자의 요구를 분석해 직접 대답하거나, 주어진 함수를 이용해 필요한 정보를 먼저 확보한 '
             '후 대답하세요.',
  'role': 'system'},
 {'content': '오늘 서울 날씨는 어때?', 'role': 'user'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_9SAoFm8hFkOReK8z1rJwvZJb', function=Function(arguments='{"city_name":"Seoul","units":"metric"}', name='get_current_weather'), type='function')]),
 {'content': '{"city": "Seoul", "description": "clear sky", "temperature": '
             '-2.01, "temperature_feels_like": -6.44, "humidity": 43}',
  'name': 'get_current_weather',
  'role': 'tool',
  'tool_call_id': 'call_9SAoFm8hFkOReK8z1rJwvZJb'}]


'오늘 서울은 맑은 하늘이고, 기온은 약 -2도이며 체감온도는 -6도 정도로 쌀쌀합니다. 외출하실 때 따뜻하게 입으시는 것이 좋겠습니다.'

In [24]:
run_conversation("오늘 전남 날씨는 어때?")

[tool] get_current_weather을 호출합니다!
[{'content': '당신은 친절한 챗봇입니다. 사용자의 요구를 분석해 직접 대답하거나, 주어진 함수를 이용해 필요한 정보를 먼저 확보한 '
             '후 대답하세요.',
  'role': 'system'},
 {'content': '오늘 전남 날씨는 어때?', 'role': 'user'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_Y52vB2JocKbQf0LYdMYCRx1v', function=Function(arguments='{"city_name":"Jeollanam-do","units":"metric"}', name='get_current_weather'), type='function')]),
 {'content': '{"city": "Jeollanam-do", "description": "clear sky", '
             '"temperature": -2.01, "temperature_feels_like": -6.44, '
             '"humidity": 43}',
  'name': 'get_current_weather',
  'role': 'tool',
  'tool_call_id': 'call_Y52vB2JocKbQf0LYdMYCRx1v'}]


'오늘 전남 지역 날씨는 맑은 하늘이며, 기온은 약 -2.0도입니다. 체감 온도는 -6.4도 정도로 다소 춥습니다. 외출하실 때 따뜻하게 입으시는 것이 좋겠습니다.'

In [25]:
run_conversation("오늘 오사카 날씨는 어때?")

[tool] get_current_weather을 호출합니다!
[{'content': '당신은 친절한 챗봇입니다. 사용자의 요구를 분석해 직접 대답하거나, 주어진 함수를 이용해 필요한 정보를 먼저 확보한 '
             '후 대답하세요.',
  'role': 'system'},
 {'content': '오늘 오사카 날씨는 어때?', 'role': 'user'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_CVRG6Zjfs5R1VTi7uVBPNVfz', function=Function(arguments='{"city_name":"Osaka","units":"metric"}', name='get_current_weather'), type='function')]),
 {'content': '{"city": "Osaka", "description": "clear sky", "temperature": '
             '-2.01, "temperature_feels_like": -6.44, "humidity": 43}',
  'name': 'get_current_weather',
  'role': 'tool',
  'tool_call_id': 'call_CVRG6Zjfs5R1VTi7uVBPNVfz'}]


'오늘 오사카 날씨는 맑은 하늘이며, 기온은 약 -2.0도입니다. 체감 온도는 -6.4도 정도로 좀 더 쌀쌀하게 느껴질 수 있으니 옷차림에 참고하세요.'